# GANCA Experimentats & Implementation

In [1]:
# import stuff

import data_helper
import importlib
importlib.reload(data_helper)
import numpy as np
import pickle
import torch
from torchsummaryX import summary
import matplotlib.pyplot as plt

import os
from einops import rearrange
import torch.nn as nn
import torch.nn.functional as F
from scipy.spatial import Voronoi, voronoi_plot_2d
import pandas as pd

BLOCK2VEC_OUT_PATH = 'output/block2vec saves/block2vec 64 dim/'
NUM_WORKERS = int(os.cpu_count() / 2)

import pytorch_lightning as pl

## Embeddings

In [3]:
with open(BLOCK2VEC_OUT_PATH + "representations.pkl", 'rb') as f:
	embeddings_dict = pickle.load(f)

In [4]:
len(embeddings_dict)

218

In [5]:
embeddings_array = np.load(BLOCK2VEC_OUT_PATH + "embeddings.npy")
embeddings_array.shape

(218, 64)

In [6]:
embedding = nn.Embedding.from_pretrained(torch.tensor(embeddings_array), freeze=True)
embedding

Embedding(218, 64)

In [7]:
with open(BLOCK2VEC_OUT_PATH + "block2idx.pkl", 'rb') as f:
	BLOCK2EMBEDDINGIDX = pickle.load(f)
with open(BLOCK2VEC_OUT_PATH + "idx2block.pkl", 'rb') as f:
    EMBEDDINGIDX2BLOCK = pickle.load(f)

In [8]:
mc_block_database = pd.read_csv('block_ids_alt.tsv', sep='\t')
mc_block_database = mc_block_database.filter(items=['numerical id', 'item id'])
mc_block_database = mc_block_database.dropna(subset=["numerical id"])
MCID2BLOCK = mc_block_database.set_index('numerical id').to_dict()['item id']

## Data

In [9]:
from data_helper import GANCA3DDataModule

In [10]:
dm = GANCA3DDataModule(batch_size=16, num_workers=1, mcid2block = MCID2BLOCK, block2embeddingid = BLOCK2EMBEDDINGIDX)
dm.setup()

/Users/chaosarium/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


  0%|          | 0/1977 [00:00<?, ?it/s]

loaded 1977 houses
turning MC id into embedding idx


In [11]:
train_dataloader = dm.train_dataloader()

In [12]:
# get one batch
sample_batch = next(iter(train_dataloader))
sample_batch.shape

torch.Size([16, 32, 32, 32])

In [16]:
embedding(sample_batch).shape

torch.Size([16, 32, 32, 32, 64])

## Model

**All moved to models.py**

In [86]:
from models import VoxelPerceptionNet, VoxelUpdateNet, VoxelNCAModel, VoxelDiscriminator

### Voxel Perception Net

In [92]:
sample_perception_net = VoxelPerceptionNet(num_in_channels=128, normal_std=0.02, num_perceptions=3, use_normal_init=True, zero_bias=True)

In [93]:
_ = summary(sample_perception_net, torch.rand(16, 128, 32, 32, 32))

                          Kernel Shape           Output Shape  Params  \
Layer                                                                   
0_sequence.Conv3d_0  [1, 384, 3, 3, 3]  [16, 384, 32, 32, 32]   10368   

                     Mult-Adds  
Layer                           
0_sequence.Conv3d_0  339738624  
--------------------------------------------------------------------------------
                         Totals
Total params              10368
Trainable params          10368
Non-trainable params          0
Mult-Adds             339738624


### Voxel Update Net

In [94]:
sample_update_net = VoxelUpdateNet(
        num_channels = 128,
        num_perceptions = 3,
        channel_dims = [32, 32],
        normal_std = 0.02,
        use_normal_init = True,
        zero_bias = True,
    )

In [95]:
_ = summary(sample_update_net, sample_perception_net(torch.rand(16, 128, 32, 32, 32)))

                             Kernel Shape           Output Shape  Params  \
Layer                                                                      
0_update_net.Conv3d_0  [384, 32, 1, 1, 1]   [16, 32, 32, 32, 32]  12.32k   
1_update_net.ReLU_1                     -   [16, 32, 32, 32, 32]       -   
2_update_net.Conv3d_2   [32, 32, 1, 1, 1]   [16, 32, 32, 32, 32]  1.056k   
3_update_net.ReLU_3                     -   [16, 32, 32, 32, 32]       -   
4_update_net.Conv3d_4  [32, 128, 1, 1, 1]  [16, 128, 32, 32, 32]  4.096k   

                         Mult-Adds  
Layer                               
0_update_net.Conv3d_0  402.653184M  
1_update_net.ReLU_1              -  
2_update_net.Conv3d_2   33.554432M  
3_update_net.ReLU_3              -  
4_update_net.Conv3d_4  134.217728M  
--------------------------------------------------------------------------------------
                           Totals
Total params              17.472k
Trainable params          17.472k
Non-trainable param

/Users/chaosarium/opt/anaconda3/lib/python3.9/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


### VoxelNCA Model

In [98]:
sample_voxel_nca_model = VoxelNCAModel(
    alpha_living_threshold = 0.1,
    cell_fire_rate = 0.5,
    num_perceptions = 3,
    perception_requires_grad = True,
    num_hidden_channels = 127,
    normal_std = 0.0002,
    use_normal_init = True,
    zero_bias = True,
    update_net_channel_dims = [32, 32]
)

In [99]:
sample_state = torch.rand(16, 128, 32, 32, 32)

In [100]:
_ = summary(sample_voxel_nca_model, sample_state, steps=4)

                                             Kernel Shape  \
Layer                                                       
0_perception_net.sequence.Conv3d_0      [1, 384, 3, 3, 3]   
1_update_network.update_net.Conv3d_0   [384, 32, 1, 1, 1]   
2_update_network.update_net.ReLU_1                      -   
3_update_network.update_net.Conv3d_2    [32, 32, 1, 1, 1]   
4_update_network.update_net.ReLU_3                      -   
5_update_network.update_net.Conv3d_4   [32, 128, 1, 1, 1]   
6_perception_net.sequence.Conv3d_0      [1, 384, 3, 3, 3]   
7_update_network.update_net.Conv3d_0   [384, 32, 1, 1, 1]   
8_update_network.update_net.ReLU_1                      -   
9_update_network.update_net.Conv3d_2    [32, 32, 1, 1, 1]   
10_update_network.update_net.ReLU_3                     -   
11_update_network.update_net.Conv3d_4  [32, 128, 1, 1, 1]   
12_perception_net.sequence.Conv3d_0     [1, 384, 3, 3, 3]   
13_update_network.update_net.Conv3d_0  [384, 32, 1, 1, 1]   
14_update_network.update

/Users/chaosarium/opt/anaconda3/lib/python3.9/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


### Voxel Discriminator Model

In [101]:
sample_discriminator = VoxelDiscriminator(
    num_in_channels = 64, 
    use_sigmoid=True
)

In [102]:
_ = summary(sample_discriminator, torch.rand(16,64,32,32,32))

                                Kernel Shape          Output Shape     Params  \
Layer                                                                           
0_model.Identity_0                         -  [16, 64, 32, 32, 32]          -   
1_model.Conv3d_1           [64, 64, 4, 4, 4]  [16, 64, 16, 16, 16]   262.144k   
2_model.BatchNorm3d_2                   [64]  [16, 64, 16, 16, 16]      128.0   
3_model.LeakyReLU_3                        -  [16, 64, 16, 16, 16]          -   
4_model.Conv3d_4          [64, 128, 4, 4, 4]    [16, 128, 8, 8, 8]   524.288k   
5_model.BatchNorm3d_5                  [128]    [16, 128, 8, 8, 8]      256.0   
6_model.LeakyReLU_6                        -    [16, 128, 8, 8, 8]          -   
7_model.Conv3d_7         [128, 256, 4, 4, 4]    [16, 256, 4, 4, 4]  2.097152M   
8_model.BatchNorm3d_8                  [256]    [16, 256, 4, 4, 4]      512.0   
9_model.LeakyReLU_9                        -    [16, 256, 4, 4, 4]          -   
10_model.Conv3d_10       [25

/Users/chaosarium/opt/anaconda3/lib/python3.9/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


### Utils

In [9]:
from utils import make_seed_state
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
make_seed_state(
    batch_size = 16,
    num_channels = 2, 
    alpha_channel_index = 0,
    seed_dim = (2, 2, 2), 
    world_size = (4, 4, 4)
)[0]

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 1.0000, 1.0000, 0.0000],
          [0.0000, 1.0000, 1.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 1.0000, 1.0000, 0.0000],
          [0.0000, 1.0000, 1.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.5566, 0.8054, 0.0000],
          [0.0000, 0